In [2]:
import cv2
import numpy as np
from collections import deque
class TennisCourt:
    """
    This class models the tennis court from a warped, top-down perspective.
    It processes detected lines to identify the key court lines and intersections.
    """
    def __init__(self, width, height, buffer_size=3):
        self.width = width
        self.height = height
        # Buffers to store recent line positions for stabilization
        self.h_line_buffer = deque(maxlen=buffer_size)
        self.v_line_buffer = deque(maxlen=buffer_size)

        # Stores the final, averaged court lines
        self.lines = {
            "horizontal": [],
            "vertical": []
        }
        self.intersections = []

    def process_frame(self, warped_edges):
        """
        Main processing function to find and model the court.
        """
        
        lines = cv2.HoughLinesP(warped_edges, 1, np.pi / 180, 50, minLineLength=40, maxLineGap=15)
        if lines is None:
            #self._find_intersections()
            return # No lines detected, do nothing

        
        horizontal, vertical = self._classify_lines(lines)

        
        current_h_lines = self._merge_and_average_lines(horizontal, 'horizontal')
        current_v_lines = self._merge_and_average_lines(vertical, 'vertical')
        
        if current_h_lines:
            self.h_line_buffer.append(current_h_lines)
        if current_v_lines:
            self.v_line_buffer.append(current_v_lines)
            
        self.lines["horizontal"] = self._get_stabilized_lines('horizontal')
        self.lines["vertical"] = self._get_stabilized_lines('vertical')
        
        
        self._find_intersections()

    def _get_stabilized_lines(self, orientation):
        """Averages the lines in the buffer to get a stable position."""
        buffer = self.h_line_buffer if orientation == 'horizontal' else self.v_line_buffer
        if not buffer:
            return []

        # Get all line lists from the buffer
        all_lines_from_buffer = list(buffer)
        
        # Use the number of lines from the most recent frame as the target
        num_lines = len(all_lines_from_buffer[-1])
        avg_lines = []

        # For each line index (e.g., 0=top, 1=middle), average its position over the buffer
        for i in range(num_lines):
            # Collect the i-th line from each frame in the buffer that has it
            line_group = [frame_lines[i] for frame_lines in all_lines_from_buffer if len(frame_lines) > i]
            
            if not line_group:
                continue
            
            if orientation == 'horizontal':
                # Average the y-coordinate
                avg_y = int(np.mean([line[1] for line in line_group]))
                avg_lines.append([0, avg_y, self.width, avg_y])
            else:  # vertical
                # Average the x-coordinate
                avg_x = int(np.mean([line[0] for line in line_group]))
                avg_lines.append([avg_x, 0, avg_x, self.height])
                
        # Ensure lines remain sorted by position
        coord_index = 1 if orientation == 'horizontal' else 0
        avg_lines.sort(key=lambda line: line[coord_index])

        return avg_lines

    def _classify_lines(self, lines):
        """Classify lines into horizontal and vertical from a warped perspective."""
        horizontal = []
        vertical = []
        for line in np.squeeze(lines):
            x1, y1, x2, y2 = line
            # In the warped image, lines are almost perfectly H or V
            if abs(y2 - y1) < 2:  # Horizontal
                horizontal.append(line)
            elif abs(x2 - x1) < 2:  # Vertical
                vertical.append(line)
        return horizontal, vertical

    def _merge_and_average_lines(self, lines, orientation):
        """Group close lines and average them into a single line."""
        if not lines:
            return []

        # Sort lines by their position (y for horizontal, x for vertical)
        coord_index = 1 if orientation == 'horizontal' else 0
        lines.sort(key=lambda line: line[coord_index])

        merged_lines = []
        while lines:
            base_line = lines.pop(0)
            group = [base_line]
            
            # Find all other lines in the list that are close to the base_line
            remaining_lines = []
            for line in lines:
                if abs(line[coord_index] - base_line[coord_index]) < 2: # Proximity threshold
                    group.append(line)
                elif abs(line[coord_index] - base_line[coord_index]) < 50:
                    ignore = True
                else:
                    remaining_lines.append(line)
            lines = remaining_lines

            # Average the lines in the group to a single idealized line
            if orientation == 'horizontal':
                all_y = np.concatenate([l[1::2] for l in group]) # Get all y-coordinates
                avg_y = int(np.mean(all_y))
                # Create a full-width line
                merged_lines.append([0, avg_y, self.width, avg_y])
            else: # 'vertical'
                all_x = np.concatenate([l[0::2] for l in group]) # Get all x-coordinates
                avg_x = int(np.mean(all_x))
                # Create a full-height line
                merged_lines.append([avg_x, 0, avg_x, self.height])
        
        return merged_lines

    def _find_intersections(self):
        """Calculate intersection points from the final horizontal and vertical lines."""
        self.intersections = []
        for h_line in self.lines['horizontal']:
            y = h_line[1]
            for v_line in self.lines['vertical']:
                x = v_line[0]
                self.intersections.append((x, y))

    def draw(self, image):
        """Draws the modeled court lines and intersections on an image."""
        # Draw the idealized lines in yellow
        for line_type in self.lines.values():
            for line in line_type:
                x1, y1, x2, y2 = line
                cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 1)
        
        # Draw the intersections as red circles
        for point in self.intersections:
            cv2.circle(image, point, 2, (0, 0, 255), -1)

# --- Main Video Processing Loop ---

video_path = "VideoInput/video_input2.mp4"
cap = cv2.VideoCapture(video_path)

# Your source points for perspective transform
src_points = np.float32([[288.0, 152.0], [668.0, 150.0], [182.0, 429.0], [783.0, 428.0]])
width, height = 400, 500
dst_points = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
M = cv2.getPerspectiveTransform(src_points, dst_points)

# Initialize the court detector
court_detector = TennisCourt(width, height)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (960, 540))

    # 1. Get binary edge image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # 2. Warp the edge image to get a top-down view
    warped_edges = cv2.warpPerspective(edges, M, (width, height))

    # 3. Process this frame to find and model the court
    court_detector.process_frame(warped_edges)

    # 4. Create an image to draw the detected court model on
    # We can overlay this on the warped video frame
    warped_frame = cv2.warpPerspective(frame, M, (width, height))
    court_drawing = warped_frame.copy() # Draw on a copy of the warped frame
    court_detector.draw(court_drawing)
    
    # Show the original frame and the result
    cv2.imshow("Original Frame", frame)
    cv2.imshow("Detected Court with Intersections", court_drawing)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Processing complete.")

Processing complete.


In [10]:
import cv2
import numpy as np
from collections import deque
class TennisCourt:
    """
    This class models the tennis court from a warped, top-down perspective.
    It processes detected lines to identify the key court lines and intersections.
    """
    def __init__(self, width, height, alpha=0.3):
        self.width = width
        self.height = height
        self.alpha = alpha 
        # Buffers to store recent line positions for stabilization
        self.last_h_lines = []
        self.last_v_lines = []

        # Stores the final, averaged court lines
        self.lines = {
            "horizontal": [],
            "vertical": []
        }
        self.intersections = []

    def process_frame(self, warped_edges):
        """
        Main processing function to find and model the court.
        """
        
        lines = cv2.HoughLinesP(warped_edges, 1, np.pi / 180, 50, minLineLength=50, maxLineGap=20)
        if lines is not None:
            horizontal, vertical = self._classify_lines(lines)
            
            current_h_lines = self._merge_and_average_lines(horizontal, 'horizontal')
            current_v_lines = self._merge_and_average_lines(vertical, 'vertical')

            # Smooth horizontal lines
            if self.last_h_lines and len(current_h_lines) == len(self.last_h_lines):
                for i in range(len(current_h_lines)):
                    # Apply exponential moving average
                    old_y = self.last_h_lines[i][1]
                    new_y = current_h_lines[i][1]
                    avg_y = int(old_y * (1 - self.alpha) + new_y * self.alpha)
                    self.last_h_lines[i] = [0, avg_y, self.width, avg_y]
            else:
                self.last_h_lines = current_h_lines

            # Smooth vertical lines
            if self.last_v_lines and len(current_v_lines) == len(self.last_v_lines):
                for i in range(len(current_v_lines)):
                    old_x = self.last_v_lines[i][0]
                    new_x = current_v_lines[i][0]
                    avg_x = int(old_x * (1 - self.alpha) + new_x * self.alpha)
                    self.last_v_lines[i] = [avg_x, 0, avg_x, self.height]
            else:
                self.last_v_lines = current_v_lines

        # Use the smoothed lines for drawing
        self.lines["horizontal"] = self.last_h_lines
        self.lines["vertical"] = self.last_v_lines
        
        self._find_intersections()

    def _classify_lines(self, lines):
        """Classify lines into horizontal and vertical from a warped perspective."""
        horizontal = []
        vertical = []
        for line in np.squeeze(lines):
            x1, y1, x2, y2 = line
            # In the warped image, lines are almost perfectly H or V
            if abs(y2 - y1) < 5:  # Horizontal
                horizontal.append(line)
            elif abs(x2 - x1) < 5:  # Vertical
                vertical.append(line)
        return horizontal, vertical

    def _merge_and_average_lines(self, lines, orientation):
        """Group close lines and average them into a single line."""
        if not lines:
            return []

        # Sort lines by their position (y for horizontal, x for vertical)
        coord_index = 1 if orientation == 'horizontal' else 0
        lines.sort(key=lambda line: line[coord_index])

        merged_lines = []
        while lines:
            base_line = lines.pop(0)
            group = [base_line]
            
            # Find all other lines in the list that are close to the base_line
            remaining_lines = []
            for line in lines:
                if abs(line[coord_index] - base_line[coord_index]) < 5: # Proximity threshold
                    group.append(line)
                elif abs(line[coord_index] - base_line[coord_index]) < 100:
                    ignore = True
                else:
                    remaining_lines.append(line)
            lines = remaining_lines

            # Average the lines in the group to a single idealized line
            if orientation == 'horizontal':
                all_y = np.concatenate([l[1::2] for l in group]) # Get all y-coordinates
                avg_y = int(np.mean(all_y))
                # Create a full-width line
                merged_lines.append([0, avg_y, self.width, avg_y])
            else: # 'vertical'
                all_x = np.concatenate([l[0::2] for l in group]) # Get all x-coordinates
                avg_x = int(np.mean(all_x))
                # Create a full-height line
                merged_lines.append([avg_x, 0, avg_x, self.height])
        
        return merged_lines

    def _find_intersections(self):
        """Calculate intersection points from the final horizontal and vertical lines."""
        self.intersections = []
        for h_line in self.lines['horizontal']:
            y = h_line[1]
            for v_line in self.lines['vertical']:
                x = v_line[0]
                self.intersections.append((x, y))
        

    def draw(self, image):
        """Draws the modeled court lines and intersections on an image."""
        # Draw the idealized lines in yellow
        for line_type in self.lines.values():
            for line in line_type:
                x1, y1, x2, y2 = line
                cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 1)
        
        # Draw the intersections as red circles
        for point in self.intersections:
            cv2.circle(image, point, 2, (0, 0, 255), -1)

# --- Main Video Processing Loop ---

video_path = "VideoInput/video_input2.mp4"
cap = cv2.VideoCapture(video_path)

# Your source points for perspective transform
src_points = np.float32([[288.0, 152.0], [668.0, 150.0], [182.0, 429.0], [783.0, 428.0]])
width, height = 400, 500
dst_points = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
M = cv2.getPerspectiveTransform(src_points, dst_points)

# Initialize the court detector
court_detector = TennisCourt(width, height)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (960, 540))

    # 1. Get binary edge image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # 2. Warp the edge image to get a top-down view
    warped_edges = cv2.warpPerspective(edges, M, (width, height))

    # 3. Process this frame to find and model the court
    court_detector.process_frame(warped_edges)

    # 4. Create an image to draw the detected court model on
    # We can overlay this on the warped video frame
    warped_frame = cv2.warpPerspective(frame, M, (width, height))
    court_drawing = warped_frame.copy() # Draw on a copy of the warped frame
    court_detector.draw(court_drawing)
    
    # Show the original frame and the result
    cv2.imshow("Original Frame", frame)
    cv2.imshow("Detected Court with Intersections", court_drawing)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Processing complete.")

Processing complete.


In [ ]:
def calculate_pixel_error(p1, p2):
    """Calculates the Euclidean distance between two points."""
    return np.linalg.norm(np.array(p1) - np.array(p2))

In [ ]:
errors = []
GROUND_TRUTH_FILE = "ground_truth.json"
if errors:
    mean_error = np.mean(errors)
    std_dev = np.std(errors)
    max_error = np.max(errors)
    
    print("\n--- Model Accuracy Report ---")
    print(f"Frames Evaluated: {len(errors) // 4}")
    print(f"Mean Pixel Error (MPE): {mean_error:.2f} pixels")
    print(f"Standard Deviation: {std_dev:.2f} pixels")
    print(f"Maximum Error: {max_error:.2f} pixels")
    print("---------------------------")
else:
    print("Evaluation could not be completed.")